# TC impact on Cairns - the benefit of building to code

It hasn't yet happened. But one day, maybe this year - maybe next, Cairns will be struck by a major TC. Two severe TCs (_Larry_ and _Yasi_) have hit the coast just south of Cairns in the past 10 years, and _Ita_ was close to sweeping down the coast from Cooktown, which would have been devastating for the city of over 150,000 people. 

Here we present a scenario of just such an event. The scenario was generated by GA's Tropical Cyclone Risk Model (TCRM) - a statistical-parametric model of tropical cyclones. The scenario isn't a forecast of what will happen at some point in the future - rather, it's an event that is plausible based on the observations of past TCs in the region. 

In [1]:
%matplotlib inline

from __future__ import division, print_function

from os.path import join as pjoin
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt, gridspec, cm
sns.set_context("poster")
sns.set_style('ticks')

The simulation produces a wind field across the landscape that accounts for the local effects such as topography, changing land cover and the protective shielding that buildings provide to other buildings downwind. 

The amount of damage sustained by buildings is defined by a set of vulnerability curves. For a given wind speed, these describe the level of damage as a fraction of the total replacement cost of the building. 

In this analysis, we've used three different vulnerability curves. The first is for all buildings constructed after 1982 - coinciding with the implementation of new building codes that defined specific 'cyclonic' regions around the northern Australian coastline. These new building codes were an outcome of the impacts of Cyclone _Tracy_ in Darwin (1974) and Cyclone _Ada_ in Townsville (1973) where deficient construction standards were identified as a contributing factor to the widespread destruction from _Tracy_. 

The second curve represents all buildings built prior to the code (i.e. pre-1981). These were generally more vulnerable to wind damage, with weaker connections between roof materials ans rafters and tie-downs between roof structures and the walls. 


In [2]:
datapath = "C:/WorkSpace/data/TC/scenario"

originalFile = pjoin(datapath, "wind_impact_prepost_classified.csv")
retrofitFile = pjoin(datapath, "wind_impact_retrofit_classified.csv")

originaldf = pd.read_csv(originalFile, sep=",",header=0, index_col=0, skipinitialspace=True)
retrofitdf = pd.read_csv(retrofitFile, sep=",",header=0, index_col=0, skipinitialspace=True)
retrofitdf.info()

In [3]:
def plotVulnCurve(df, buildingClass):
    if buildingClass not in df.columns[1:]:
        return None
    ws = df["Class"][2:]
    vuln = df[buildingClass][2:]
    plt.plot(ws, vuln, label=buildingClass)
    plt.xlim(0, 100)
    plt.xlabel("Wind speed (m/s)")
    plt.ylim(0, 1)
    plt.ylabel("Damage index")
    plt.title("Vulnerability curve for {0}".format(buildingClass))
    
curveFile = pjoin(datapath, "cairns_wind_vul_curves.csv")
curvedf = pd.read_csv(curveFile, sep=",",header=0, skip_blank_lines=True)
plotVulnCurve(curvedf, 'Pre 1980 north Qld house')


In [4]:
print("Original loss: ${0:,.2f}".format(np.round(np.sum(originaldf['structural_loss']))))
print("Retrofitted loss: ${0:,.2f}".format(np.round(np.sum(retrofitdf['structural_loss']))))


So the total structural loss from the cyclone, with the present building stock, would be nearly half a billion dollars - and that's only the residential buildings. Don't forget the public, commercial and industrial buildings as well as other infrastructure. That's also only the damage due to the severe winds. There'd likely be significant damage due to flooding across the city, both from riverine flooding along the Mulgrave & Barron Rivers, and storm surge on the shorefront.

If it were possible to retrofit all the older structures to bring them closer to the current code, the structural losses would fall to under $200 million. Remember - this retrofitting would only affect the damage to around one third of homes in the Cairns region, but it reduces the costs by over 60%. 

In [5]:
ax = sns.boxplot(x="WALL_TYPE", y="structural_loss_ratio", data=originaldf, palette=sns.color_palette("RdBu_r", 4))
ax.set_ylim(0, 1)
ax.set_xlabel("Wall type")
ax.set_ylabel('Structural loss ratio')
ax.set_title('Original building stock')

In [6]:
ax = sns.boxplot(x="WALL_TYPE", y="structural_loss_ratio", data=retrofitdf, palette=sns.color_palette("RdBu_r", 4))
ax.set_ylim(0, 1)
ax.set_xlabel("Wall type")
ax.set_ylabel('Structural loss ratio')
ax.set_title('Retrofitted building stock')

In [7]:
age_order = ['1840 - 1890', '1891 - 1913', '1914 - 1946', 
             '1947 - 1961', '1962 - 1981', '1982 - 1996', 
             '1997 - present']
ax = sns.boxplot(x="YEAR_BUILT", y="structural_loss_ratio", data=originaldf, 
                 order=age_order, palette=sns.color_palette("RdBu_r", 7))
ax.set_ylim(0,1)
ax.set_xlabel("Year built")
ax.set_ylabel('Structural loss ratio')
ax.set_title('Original building stock')

In [8]:
ax = sns.boxplot(x="YEAR_BUILT", y="structural_loss_ratio", data=retrofitdf, 
                 order=age_order, palette=sns.color_palette("RdBu_r", 7))
ax.set_ylim(0, 1)
ax.set_xlabel("Year built")
ax.set_ylabel('Structural loss ratio')
ax.set_title('Retrofitted building stock')

Understandably the biggest change in losses is in the older building stock - that's what is being modified. Losses to homes built between 1982 and the present are unchanged. But the losses for post-war construction fall from an average of around 5% to below 1%. In fact, the average loss for older building stock becomes less than for the 1997 - present era buildings. What might be going on here?

Location, location, location. The newer houses are being constructed in more exposed locations - e.g. on elevated terrain near the outer parts of the city. Suburbs like Smithfield, Caravonica, Redlynch, Edmonton and White Rock ar largely new builds, but are creeping up the slopes of the Great Dividing Range, which exacerbates the winds. Those suburbs on the southern side of Cairns are worst affected, as this is where the band of strongest winds passes. 

In [9]:
elevFile = pjoin(datapath, "wind_impact_prepost_elev.csv")
elevdf = pd.read_csv(elevFile, sep=",",header=0, index_col=0, skipinitialspace=True)

In [10]:
ax = sns.boxplot(x="YEAR_BUILT", y="elevation", data=elevdf, 
                 order=age_order, palette=sns.color_palette("RdBu", 7),
                 medianprops={"color":'darkred'})
ax.set_xlabel("Year built")
ax.set_ylabel('Elevation')
ax.set_ylim(0, 350)
ax.set_title("Elevation by building age")

Notice the mean value increasing steadily through the building age classes. Early settlement was on the flat areas around near the city centre and in suburbs like Edge Hill, Stratford and some of the smaller beachside suburbs to the north Trinity Beach, Yorkey's Knob, Holloways Beach, Machans Beach). 

Newer suburbs have been established progressively further inland and encroached onto the hills surrounding Cairns. This means the houses are exposed to higher wind speeds, due to the speed up of winds over the hills and ridges. Under present building codes, construction on these more elevated sites must account for the local enhancements of wind speed (under AS4055 - Wind loads for housing). In this analysis we have used a single vulnerability model for all classes, which results in an overestimate of the damage to newer construction. 

In [11]:
ax = sns.boxplot(x="SUBURB_2015", y="structural_loss_ratio", data=retrofitdf)
ax.set_ylim(0, 1)
ax.set_xlabel("Suburb")
plt.xticks(rotation='vertical')
ax.set_ylabel('Loss ratio')

In [12]:
byage = originaldf.groupby(['YEAR_BUILT'])
byage['structural_loss'].sum()

In [13]:
bysuburb = originaldf.groupby(['SUBURB_2015'])
bysubloss = bysuburb['structural_loss_ratio'].mean()

In [14]:
byage = elevdf.groupby('YEAR_BUILT')
byage['elevation'].mean()


In [15]:
byage['elevation'].quantile(0.75)

In [16]:
bysub = elevdf.groupby(["SUBURB_2015"])
bysubloss = bysub['structural_loss_ratio'].mean()
bysubelev = bysub['elevation'].mean()
ax = sns.regplot(x=bysubelev, y=bysubloss, order=2)
ax.set_xlim((0,120))
ax.set_ylim((0,0.1))

In [17]:
sns.jointplot('elevation', 'structural_loss_ratio', data=elevdf, kind='scatter', stat_func=None,
               joint_kws={"alpha":0.2}, xlim=(0, 350), ylim=(0, 1))

In [18]:
byage = originaldf.groupby('YEAR_BUILT')
x1 = byage['structural_loss_ratio'].mean()
retrofitbyage = retrofitdf.groupby('YEAR_BUILT')
x2 = retrofitbyage['structural_loss_ratio'].mean()
idx = x1.index
d1 = pd.DataFrame({'original':pd.Series(x1, index=idx), 'retrofit':pd.Series(x2, index=idx)})

x1 = byage['structural_loss'].sum()
retrofitbyage = retrofitdf.groupby('YEAR_BUILT')
x2 = retrofitbyage['structural_loss'].sum()
idx = x1.index
d2 = pd.DataFrame({'original':pd.Series(x1, index=idx), 'retrofit':pd.Series(x2, index=idx)})

In [19]:
def autolabel(rects, rotation='horizontal'):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        if np.isnan(height):
            height = 0
        ax.text(rect.get_x()+rect.get_width()/2., 1.05*height, '%d'%int(height),
                ha='center', va='bottom', rotation=rotation)

In [20]:
fig,axes=plt.subplots(2,1, sharex=True)
rects = axes[0].bar(np.arange(7)-.35, d1.original, width=0.35, color='#4775D1', label='Original')
rects = axes[0].bar(np.arange(7), d1.retrofit, width=0.35, color='#FF3333', label='Retrofitted')
axes[0].set_xticklabels(('',) + tuple(d1.index))
axes[0].set_ylabel('Structural loss ratio')
axes[0].legend(loc=1)
autolabel(axes[0].patches)

rects = axes[1].bar(np.arange(7)-.35, d2.original/10**6, width=0.35, color='#4775D1', label='Original')
rects = axes[1].bar(np.arange(7), d2.retrofit/10**6, width=0.35, color='#FF3333', label='Retrofitted')
axes[1].set_xticklabels(('',) + tuple(d2.index))
axes[1].set_ylabel('Structural loss (A$ million)')
axes[1].legend(loc=1)
autolabel(rects)



We expect no change in the losses for the 1982 - 1996 and 1997 - present classes, since these houses are expected to be built to a reasonable standard (arguably one that is appropriate for the site). 

The retrofitting of older housing sees the average loss ratio across the older buildings decrease dramatically. 

When we look at the financial benefit of retrofitting, the greatest gains are made in the 1962 - 1981 class. Costs fall to 16% of the unmodified case. All the older building classes see a similar percentage fall in costs, but because of the significant amount of housing stock in this age range, we see a huge drop in the total costs. 

Of course the trade off is that there is a cost associated with retrofitting the entire housing stock in a city. 

In [21]:
d2['retrofit'] / d2['original']

In [22]:
print(d2.to_string(formatters={'original':'${:,.2f}'.format,
                               'retrofit':'${:,.2f}'.format}))

In [23]:
def lossCat(df):
    """Add a loss category to the given data frame"""
    categoryNames = ['Negligible', 'Slight', 'Moderate', 'Extensive', 'Complete']
    categoryThres = [0.0, 0.01, 0.1, 0.2, 0.5]
    df['structural_loss_category'] = 'Negligible'
    for thres, cat in zip(categoryThres, categoryNames):
        idx = df['structural_loss_ratio'] > thres
        df['structural_loss_category'].loc[idx] = cat
    return df

lossCat(originaldf)
lossCat(retrofitdf)

In [24]:
ax = sns.countplot(x='structural_loss_category', data=originaldf, hue='YEAR_BUILT', palette='RdBu', 
                   order=['Negligible', 'Slight', 'Moderate', 'Extensive', 'Complete'])
autolabel(ax.patches, rotation='vertical')
ax.set_title("Original building stock")
ax.set_xlabel("Structural loss category")
ax.legend(loc=1)

In [25]:
originaldf['structural_loss_category'].value_counts()

In [26]:
ax = sns.countplot(x='structural_loss_category', data=originaldf, palette='RdBu', 
                   order=['Negligible', 'Slight', 'Moderate', 'Extensive', 'Complete'])
autolabel(ax.patches)
ax.set_xlabel("Structural loss category")

In [27]:
retrofitdf['structural_loss_category'].value_counts()

In [28]:
ax = sns.countplot(x='structural_loss_category', data=retrofitdf, palette='RdBu', 
                   order=['Negligible', 'Slight', 'Moderate', 'Extensive', 'Complete'])
autolabel(ax.patches)
ax.set_xlabel("Structural loss category")

In [29]:
ax = sns.countplot(x='structural_loss_category', data=retrofitdf, hue='YEAR_BUILT', palette='RdBu', 
                   order=['Negligible', 'Slight', 'Moderate', 'Extensive', 'Complete'])
autolabel(ax.patches, rotation='vertical')
ax.set_title("Retrofitted building stock")
ax.set_xlabel("Structural loss category")
ax.legend(loc=1)

In [30]:
x = originaldf['YEAR_BUILT'].value_counts()
plt.pie(x, labels=x.index, colors=sns.color_palette("RdBu", 7), 
        explode=100./x+.01,
        autopct='%.2f%%', shadow=True, wedgeprops={'linewidth':2, 'ec':'0.75'})
plt.axis('equal')
plt.title("Building age distribution")

Here we categorise by broader age groups. We group together those houses built prior to the end of WWII (1946), those built between the end of WWII and the implementation of strict building codes in 1981, and those built after 1981.

In [31]:
def ageCat(df):
    """Add broad age category to the given data frame"""
    categoryNames = ['1840 - 1946', '1946 - 1981', '1982 - present']
    categoryThres = [['1840 - 1890', '1891 - 1913', '1914 - 1946'], 
                     ['1947 - 1961', '1962 - 1981'],
                     ['1982 - 1996', '1997 - present']]
    df['age_category'] = '1840 - 1946'
    for thres, cat in zip(categoryThres, categoryNames):
        df['age_category'].loc[df['YEAR_BUILT'].isin(thres)] = cat
    return df
                     
ageCat(originaldf)
ageCat(retrofitdf)

In [32]:
byagecat = originaldf.groupby('age_category')
x1 = byagecat['structural_loss_ratio'].mean()
retrofitbyagecat = retrofitdf.groupby('age_category')
x2 = retrofitbyagecat['structural_loss_ratio'].mean()
idx = x1.index
d1 = pd.DataFrame({'original':pd.Series(x1, index=idx), 'retrofit':pd.Series(x2, index=idx)})

x1 = byagecat['structural_loss'].sum()
retrofitbyagecat = retrofitdf.groupby('age_category')
x2 = retrofitbyagecat['structural_loss'].sum()
idx = x1.index
d2 = pd.DataFrame({'original':pd.Series(x1, index=idx), 'retrofit':pd.Series(x2, index=idx)})

In [33]:
d2['retrofit'] / d2['original']

In [34]:
d2.index

In [35]:
fig,axes=plt.subplots(1,1, sharex=True)
rects = axes.bar(np.array([0, 1, 2])-.35, d1.original, width=0.35, color='#4775D1', label='Original')
rects = axes.bar(np.array([0, 1, 2]), d1.retrofit, width=0.35, color='#FF3333', label='Retrofitted')
axes.set_xticklabels(['', '1840 - 1946', '', '1947 - 1981', '', '1982 - present'])
axes.set_ylim(0, 0.05)
axes.set_ylabel('Damage ratio')
axes.legend(loc=1)
autolabel(axes.patches)

fig,axes=plt.subplots(1,1, sharex=True)
rects = axes.bar(np.array([0, 1, 2])-.35, d2.original/10**6, width=0.35, color='#4775D1', label='Original')
rects = axes.bar(np.array([0, 1, 2]), d2.retrofit/10**6, width=0.35, color='#FF3333', label='Retrofitted')
axes.set_xticklabels(['', '1840 - 1946', '', '1947 - 1981', '', '1982 - present'])
axes.set_ylabel('Structural loss (A$ million)')
axes.legend(loc=1)
autolabel(rects)

In [36]:
estimator = lambda x: np.percentile(x, 95)

age_order = [u'1840 - 1946', u'1946 - 1981', u'1982 - present']
ax = sns.barplot(x="age_category", y="structural_loss_ratio", data=originaldf, 
                 order=age_order, palette=sns.color_palette("RdBu_r", 3), ci=None)

ax.set_ylim(0,0.1)
ax.set_xlabel("Year built")
ax.set_ylabel('Mean damage ratio')
ax.set_title('Original building stock')

In [37]:
age_order = [u'1840 - 1946', u'1946 - 1981', u'1982 - present']
ax = sns.barplot(x="age_category", y="structural_loss_ratio", data=retrofitdf,
                 order=age_order, palette=sns.color_palette("RdBu_r", 3), ci=None)
ax.set_ylim(0,0.05)
ax.set_xlabel("Year built")
ax.set_ylabel('Mean damage ratio')
ax.set_title('Retrofitted building stock')

In [41]:
bysub = originaldf.groupby('SUBURB_2015')
bysub['structural_loss'].sum()/bysub['REPLACEMENT_VALUE'].sum()

In [47]:
bysub['structural_loss_ratio'].mean().values